# Comparison of all models

In this notebook: we search for optimal hyperparameters, then select the best instances of each model class for 50 train-validate trials.

### What to do...
Select your preferred training input size (TR_IN), dataset (MG17, Lorenz, Rossler, Henon), and models (from all or a subset of those provided) to use.

### Tasks and Data
* We will look at TR_IN of 250, 500, 750, 1000 for MG17; we consider 250 and 750 for Lorenz, Rossler, and Henon
* We have 24 connectome models that are new (LHR and CAR only), 5 legacy models, and 3 random ESNs

In [ ]:
#specify the number of training input samples you want to use. 
##############################################################################
TR_IN = 250
##############################################################################

# specify the dataset you wish to consider (MG17, Lorenz, Rossler, Henon).
##############################################################################
Dataset = "MG17"
##############################################################################

# specif a list of all models you wish to use (model names are above)
##############################################################################
# models size 2632
Models = ["LHR_dir_diag_nonorm","LHRrandom"]
##############################################################################

In [ ]:
# import necessary packages

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle as pkl
import random
random.seed(0)
import scipy.stats
import seaborn as sns
import sys
sys.path.insert(0, '../../easyesn-master/')
import time

from easyesn import PredictionESN
from IPython.display import display
from matplotlib import rc

In [ ]:
# read data
data = pd.read_csv(f"Dataset/{Dataset}.DAT", header=None).values.reshape(-1)

plt.plot(data[0:1000])
plt.show()

# specify training input/output and validation input/output sizes
val_size = int(TR_IN/5)
data_size = int(TR_IN*2+val_size*2)

print(f"Training input/output size is: {TR_IN}")
print(f"Validation input/output size is: {val_size}")
print(f"Total size of dataset used is: {data_size}")

x = data[:data_size]
inputDataTraining = x[:TR_IN]
outputDataTraining = x[TR_IN:TR_IN*2]
inputDataValidation = x[TR_IN*2:TR_IN*2+val_size]
outputDataValidation = x[TR_IN*2+val_size:] 

# specify error
mse = lambda y,ypr: np.mean((y-ypr)**2)

# select hyperparameter range
reg_coef = np.exp(-1*(np.linspace(3,-7,5)))
display(reg_coef)
lr_coef = np.linspace(0.1,1,7)
display(lr_coef)

### All functions used

In [ ]:
# this function initiates a hyperparameter search for a given model class
# it returns a numpy array of all mean-squared errors.
# optionally, one can retrieve predictions as well (each is stored in ypr)

# "model_name" -> corresponds to the weightGeneration argument
# "empty_mses_array" -> an empty array to store all mean-squared errors
# "reg_vals" -> a numpy array of all hyperparameter values (first hyperparam)
# "lr_vals" -> a numpy array of all hyperparameter values (second hyperparam)
# "tr/val in/out" -> correspond to training/validation input/output
def hyperparam_search(size, model_name, empty_mses_array, reg_vals, lr_vals, tr_in,
                      tr_out,val_in,val_out):
    count = 0
    for i in range(len(reg_vals)):
        for j in range(len(lr_vals)):
            count+= 1
            print("*"*60)
            print(f"Run {count} of {len(reg_vals)*len(lr_vals)} for {model_name}")
            print(f"lambda: {reg_vals[i]} | alpha: {lr_vals[j]}")
            esn = PredictionESN(n_input=1, n_output=1, n_reservoir=size, 
                                 weightGeneration=f"{model_name}", leakingRate=lr_vals[j], 
                                 regressionParameters=[reg_vals[i]], solver="lsqr", 
                                 feedback=False)
            if(TR_IN<250):
                esn.fit(tr_in, tr_out, transientTime=10)
            else:
                esn.fit(tr_in, tr_out, transientTime=100)
            ypr = esn.predict(val_in)
            model_mse = mse(val_out,ypr.reshape(-1))
            empty_mses_array[i,j] = model_mse
            print(f"mse: {model_mse}")
            print("*"*60)
    return empty_mses_array

In [ ]:
# this function finds the optimal hyperparameters for each respective model
# it returns a dictionary where the key is the model name string and the value is a 2-tuple of best 
# currently all models need the same reservoir size for this to work
def feedmodels_hyp_search(model_list, models_reservoir_size, hyperparam1_array, hyperparam2_array):
    model_mses_list = []
    model_hyperparams_dict = {}
    for model_number, model_name in enumerate(model_list):
        # we call hyperparam search for each model. We end up with a list of arrays (each array has #reg coefs * #lr coefs size)
        this_model_mses = hyperparam_search(models_reservoir_size, model_name, 
                              np.zeros((hyperparam1_array.shape[0], hyperparam2_array.shape[0])), hyperparam1_array, 
                              hyperparam2_array, inputDataTraining, outputDataTraining, inputDataValidation, 
                              outputDataValidation)
        #print(this_model_mses)
        # get the best hyperparameter values, place into the hyperparam dictionary
        best_hyperparams = np.where(this_model_mses == np.amin(this_model_mses))
        #print(best_hyperparams)
        print(f"Min. error: {np.min(this_model_mses)}")
        model_hyperparams_dict[model_name] = (hyperparam1_array[best_hyperparams[0]][0], hyperparam2_array[best_hyperparams[1]][0])
        #print(model_hyperparams_dict)

        # add these mses to the model_mses_list (as a checking measure)
        model_mses_list.append(this_model_mses)
    #returns a list of arrays of error values (corresponding to each model) and a dictionary of best hyperparams
    return model_mses_list, model_hyperparams_dict

In [ ]:
# this function runs 50 train-validate trials for a given model class
# it returns a numpy array of all mean-squared errors.

# "model_name" -> corresponds to the weightGeneration argument
# "empty_mses_array" -> an empty array to store all mean-squared errors
# "best_reg" -> the best regularization parameter value from hyperparam. search
# "best_lr" -> the best leaking rate parameter value from hyperparam. search
# "tr/val in/out" -> correspond to training/validation input/output
def train_validate_N_times(N_times, size, model_name, empty_mses_array, empty_yprs_array, 
                           best_reg, best_lr, tr_in, tr_out,val_in,val_out):  
    for i in range(N_times):
        print("*"*60)
        print(f"Run {i+1} of {N_times} for {model_name}")
        esn = PredictionESN(n_input=1, n_output=1, n_reservoir=size, 
                            weightGeneration=f"{model_name}", 
                            leakingRate=best_lr, regressionParameters=[best_reg],
                            solver="lsqr", feedback=False)
        if(TR_IN<250):
            esn.fit(tr_in, tr_out, transientTime=10)
        else:
            esn.fit(tr_in, tr_out, transientTime=100)
        ypr = esn.predict(val_in)
        empty_yprs_array[i,:] = ypr.reshape(-1)
        mse1 = mse(val_out,ypr.reshape(-1))
        empty_mses_array[i] = mse1
        print(f"mse: {mse1}")
        print("*"*60)
    return empty_mses_array, empty_yprs_array

In [ ]:
def feedmodels_train_validate(model_list, models_reservoir_size, N_times, model_hyperparams_dict,
                             tr_in, tr_out, val_in, val_out):
    model_results_dict = {} #here we store the mses and yprs for each model, key is model name
    for model_number, model_name in enumerate(model_list):
        this_model_mses_array, this_model_yprs_array = train_validate_N_times(
            N_times, models_reservoir_size, model_name, np.zeros(N_times), 
            np.zeros((N_times, inputDataValidation.shape[0])), model_hyperparams_dict[model_name][0],
            model_hyperparams_dict[model_name][1], tr_in, tr_out, val_in, val_out)
        
        model_results_dict[model_name] = (this_model_mses_array, this_model_yprs_array)
    return model_results_dict # return dictionary containing 2-tuples of mses and yprs per each model

In [ ]:
# Here we can save any results for later use...
def save_model_results(model_names, model_results_dictionary, N_runs, TR_IN):
    for i in model_names:
        mse_arr = model_results_dictionary[i][0]
        ypr_arr = model_results_dictionary[i][1]
        with open(f'Results/{Dataset}-TR_IN_{TR_IN}-{i}_mses_{N_runs}_runs.npy', 'wb') as f:
            np.save(f, mse_arr)
        with open(f'Results/{Dataset}-TR_IN_{TR_IN}-{i}_yprs_{N_runs}_runs.npy', 'wb') as f:
            np.save(f, ypr_arr) 

## Experiment 1: CAR-size reservoirs (2632, 2639)

In [ ]:
# hyperparameter search
mses4286, best_hyp4286 = feedmodels_hyp_search(Models, 4286, reg_coef, lr_coef)

mse: 0.02110822406717047
************************************************************
************************************************************
Run 20 of 35 for LHR_dir_diag_nonorm
lambda: 7.38905609893065 | alpha: 0.85
LHR_dir_diag_nonorm
[[  1 426   0 ...   0   0   0]
 [  2   1   2 ...   0   0   0]
 [  3   0   1 ...   0   0   0]
 ...
 [  0   0   0 ...   1   0   0]
 [  0   0   0 ...   0   1   0]
 [  0   0   0 ...   0   0   1]]
mse: 0.025152051779803578
************************************************************
************************************************************
Run 21 of 35 for LHR_dir_diag_nonorm
lambda: 7.38905609893065 | alpha: 1.0
LHR_dir_diag_nonorm
[[  1 426   0 ...   0   0   0]
 [  2   1   2 ...   0   0   0]
 [  3   0   1 ...   0   0   0]
 ...
 [  0   0   0 ...   1   0   0]
 [  0   0   0 ...   0   1   0]
 [  0   0   0 ...   0   0   1]]
mse: 0.027202000358943056
************************************************************
*******************************************

mse: 0.016902730090769466
************************************************************
************************************************************
Run 3 of 35 for LHRrandom
lambda: 0.049787068367863944 | alpha: 0.4
Using LHR-sized random reservoir...
[[-0.00670608  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -0.05793088]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.04390448]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
mse: 0.03103373055024857
************************************************************
************************************************************
Run 4 of 35 for LHRrandom
lambda: 0.049787068367863944 | alpha: 0.5499999999999999
Using LHR-sized random reservoir...
[[-0.00670608  0.          0.         

mse: 0.024693569190441526
************************************************************
************************************************************
Run 15 of 35 for LHRrandom
lambda: 7.38905609893065 | alpha: 0.1
Using LHR-sized random reservoir...
[[-0.00670608  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -0.05793088]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.04390448]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
mse: 0.025708131470748144
************************************************************
************************************************************
Run 16 of 35 for LHRrandom
lambda: 7.38905609893065 | alpha: 0.25
Using LHR-sized random reservoir...
[[-0.00670608  0.          0.         ...  0.          0.

mse: 0.02674908780603686
************************************************************
************************************************************
Run 27 of 35 for LHRrandom
lambda: 90.01713130052181 | alpha: 0.85
Using LHR-sized random reservoir...
[[-0.00670608  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -0.05793088]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.04390448]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
mse: 0.025957969619352688
************************************************************
************************************************************
Run 28 of 35 for LHRrandom
lambda: 90.01713130052181 | alpha: 1.0
Using LHR-sized random reservoir...
[[-0.00670608  0.          0.         ...  0.          0

In [ ]:
# train and validate best models
models4286_dict = feedmodels_train_validate(Models, 4286, 30, best_hyp4286, inputDataTraining, 
                                    outputDataTraining, inputDataValidation, outputDataValidation)

mse: 0.011722836597950204
************************************************************
************************************************************
Run 22 of 30 for LHR_dir_diag_nonorm
LHR_dir_diag_nonorm
[[  1 426   0 ...   0   0   0]
 [  2   1   2 ...   0   0   0]
 [  3   0   1 ...   0   0   0]
 ...
 [  0   0   0 ...   1   0   0]
 [  0   0   0 ...   0   1   0]
 [  0   0   0 ...   0   0   1]]
mse: 0.015029199875292644
************************************************************
************************************************************
Run 23 of 30 for LHR_dir_diag_nonorm
LHR_dir_diag_nonorm
[[  1 426   0 ...   0   0   0]
 [  2   1   2 ...   0   0   0]
 [  3   0   1 ...   0   0   0]
 ...
 [  0   0   0 ...   1   0   0]
 [  0   0   0 ...   0   1   0]
 [  0   0   0 ...   0   0   1]]
mse: 0.020133011401328643
************************************************************
************************************************************
Run 24 of 30 for LHR_dir_diag_nonorm
LHR_dir_diag_nonorm
[

mse: 0.01601097398236257
************************************************************
************************************************************
Run 9 of 30 for LHRrandom
Using LHR-sized random reservoir...
[[-0.00670608  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -0.05793088]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.04390448]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
mse: 0.023651498404151573
************************************************************
************************************************************
Run 10 of 30 for LHRrandom
Using LHR-sized random reservoir...
[[-0.00670608  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.

mse: 0.0148466472207012
************************************************************
************************************************************
Run 22 of 30 for LHRrandom
Using LHR-sized random reservoir...
[[-0.00670608  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -0.05793088]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.04390448]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
mse: 0.02186831298543642
************************************************************
************************************************************
Run 23 of 30 for LHRrandom
Using LHR-sized random reservoir...
[[-0.00670608  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0. 

In [ ]:
# plot model results
arrs1 = [models4286_dict[i][0] for i in Models] #for the first set of models
fig, ax1 = plt.subplots(nrows=1, ncols=1,figsize=(20,5),dpi=1250)

box1 = sns.boxplot(ax=ax1, data=arrs1, palette="Set2")
ax1.set_xticklabels(Models,size=10)
ax1.set_xlabel("Model",size=20)
ax1.set_ylabel("MSE",size=20)
plt.yticks(fontsize=20)
plt.show()

In [ ]:
# save model results
save_model_results(Models, models4286_dict, 30, TR_IN)
